# LangChain + Youtube
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. 기본 패키지 설치
2. Azure 계정 연결
3. Youtube 영상 불러오기
4. LLM 생성
5. 데이터 Split
6. Chain 연결


* 유사한 서비스 Lilys.ai [https://lilys.ai](https://lilys.ai)

In [1]:
# 1. 패키지 설치

%pip install pytube
%pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

# 환경 변수 설정!! 
import os
import langchain

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)
# lagchain 사용시 반드시 환경변수 값 사용
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [18]:
# 3. Youtube 영상 데이터 읽어오기

from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url("https://youtu.be/ZFELB6yKY-E?si=rjJuhQN8hLd2bQ-m")
# loader = YoutubeLoader.from_youtube_url("https://youtu.be/5WVgXjXbLEg?si=9OizeLL6tGTtI-I4",language=['ko'],add_video_info=True)
data = loader.load()

In [19]:
# 4. Azure OpenAI LLM(Chat Model 연결)

from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=512,
    temperature=0.2,
)

In [20]:
# 토큰 수 확인
chat_llm.get_num_tokens(data[0].page_content)

2100

In [22]:
# 5. Token Split

from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = splitter.split_documents(data)

In [23]:
# 사용자 질문 기반 아니라, 요약은 임베딩 과정 필요하지 않음

# 6. Summarize Chain without LCEL

from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(
    llm=chat_llm,
    chain_type="map_reduce",
    return_intermediate_steps=True
)

chain.invoke({"input_documents" : docs}, return_only_outputs=True)

{'intermediate_steps': ['Tesla is making plans to change the cathode refinery process for their self-developed 4680 cell and advancing their lithium refinery in South Texas. They plan to leverage humanoid robot Optimus in their factories to further accelerate battery production. Musk envisions a future where there is at least one robot per human. Tesla aims to produce about a billion robots per year hoping to capture at least 10% of the market. Tesla has made strides to enhance the performance of its 4680 battery and is exploring nmc 973 which might be introduced later in 2024. Tesla is expanding its battery production capabilities with strategic moves in different locations.',
  'Tesla is expanding its battery production capabilities with strategic moves in different locations, including a new facility for LFP batteries in Sparks, Nevada, and exploring the possibility of opening a battery plant in Indonesia. Other companies, such as BYD and CATL, are also advancing battery technology,

In [14]:
chain

MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['text'], template='{text}\n# 유튜브 부분 영상 자막을 한글로 요약 정리한다\n# 가독성을 높이기 위해 핵심 문장을 3~5개 리스트로 정리한다\n# 예시\n* 이 영상에서는 xx을 설명한다\n* xx이 yy되었음을 zz를 통해 알 수 있다\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f3ef8375450>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f3ef8376b60>, temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=512, azure_endpoint='https://aoai-31-cgy.openai.azure.com/', deployment_name='gpt-35-turbo', openai_api_version='2024-02-15-preview', openai_api_type='azure')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['text'], template='{text}\n# 여러 개의 자막 요약본을 종합하여 영상의 전체 내용을 한글로 요약한다\n'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f3ef8375450>, async_cl

In [24]:
# 7. Custom Prompt without LCEL

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

prompt_template = """{text}
# 유튜브 부분 영상 자막을 한글로 요약 정리한다
# 가독성을 높이기 위해 핵심 문장을 3~5개 리스트로 정리한다
# 예시
* 이 영상에서는 xx을 설명한다
* xx이 yy되었음을 zz를 통해 알 수 있다
"""

combine_prompt_template = """{text}
# 여러 개의 자막 요약본을 종합하여 영상의 전체 내용을 한글로 요약한다
"""

# map : 부분 영상 자막 각각 요약
MAP_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
# reduce : 부분 영상 요약본 총합
COMBINE_PROMPT = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

chain = load_summarize_chain(
    llm=chat_llm,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    map_prompt=MAP_PROMPT, combine_prompt=COMBINE_PROMPT)


In [25]:
chain.invoke({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': ['- 이 영상에서는 Tesla의 배터리 생산 기술 혁신과 로봇 생산 시스템에 대해 다룬다.\n- Tesla는 4680 셀의 제조 공정을 혁신하여 생산 속도와 품질을 개선하고 있다.\n- 또한, Optimus 로봇을 활용하여 배터리 생산을 가속화하고, 미국 내에서 리튬 생산 시설을 건설하고 있다.',
  '- 이번 영상에서는 Tesla의 새로운 배터리 기술과 이를 둘러싼 경쟁 상황에 대해 다룬다.\n- Tesla는 미국 내 lfp 배터리 생산 시설을 새로 설치하고, 인도네시아에 배터리 공장을 개설할 가능성도 탐구 중이다.\n- lfp 배터리 기술은 여전히 전기차 시장에서 중요한 위치를 차지하고 있으며, 중국의 BYD와 CATL 등이 6C 충전 기술을 개발하고 있다.\n- 하지만 배터리의 에너지 밀도 문제를 해결하기 위해 고체 전해질, 나트륨 등의 새로운 배터리 기술도 개발 중이다.\n- 현재는 lfp 배터리 기술을 중심으로 중국 기업들이 선두를 달리고 있으며, CATL이 시장 점유율 37%로 1위를 차지하고 있다.',
  '1. 이 영상에서는 리튬 이온 배터리 제조 업계의 현황과 중국 기업들의 선두주 위치를 다룬다.\n2. CATL, BYD, LG Energy Solution 등 중국 기업들이 시장 점유율을 높이고 있으며, 테슬라도 LFP 셀 개발과 새로운 시설 투자를 통해 배터리 제조 업계에서 큰 역할을 할 수 있을 것으로 기대된다.\n3. 테슬라가 배터리 제조 업계에서 성공할 수 있을지에 대한 의견을 나누기 위해 댓글을 남기라는 요청이 있다.'],
 'output_text': '이 영상에서는 Tesla의 배터리 생산 기술 혁신과 로봇 생산 시스템, 그리고 새로운 배터리 기술과 이를 둘러싼 경쟁 상황에 대해 다룬다. Tesla는 4680 셀의 제조 공정을 혁신하여 생산 속도와 품질을 개선하고 있으며, Optimus 로봇을 활용하여 배터리 생산을 가속화하고 있다. 또한, 미국 내에서 리튬 생산 시설을 건설하고 있으며, 

In [15]:
# 8. Summarization Chain with LCEL

from langchain_core.output_parsers import StrOutputParser

# Map Chain
map_chain= (
    MAP_PROMPT
   | chat_llm
   | StrOutputParser() )

In [16]:
final_summary = ""

for doc in docs:
   new_summary = map_chain.invoke({"text": doc.page_content})
   final_summary+=new_summary
   print("========================")
   print(new_summary)

* 이 영상에서는 최영석 국장이 월드카 워즈에 대해 설명한다
* 월드카 워즈는 전 세계 기자들이 참여하여 올해의 차를 뽑는 행사이다
* 현대차와 기아차가 최근 몇 년간 올해의 차를 수상하며 기술력이 인정받고 있다는 것을 지적한다.
* 이 영상에서는 주식 방송에서 주가 이야기를 하지 않는다는 것을 강조한다
* 미국과 유럽에서 중국산 전기차에 대한 관세 인상이 이루어지고 있는데, 독일 업체들은 이에 반대하고 있다
* 테슬라도 중국에서 생산한 자동차를 미국으로 수입할 계획이 있었기 때문에 중국 관세 인상에 반대한 것으로 보인다.
- 이 영상에서는 테슬라의 중국 생산과 미국 시장 대응에 대해 이야기한다
- 중국에서 공급 과역 문제가 있어서 테슬라도 중국에서 생산한 차량을 수출해야 할 가능성이 있다
- 전기차의 역사와 디젤 차의 인기 하락, 그리고 기후 변화 협의에 대해 이야기한다
* 이 영상에서는 기후 재앙과 탄소 중립에 대해 이야기한다
* 테슬라의 전기차가 자동차 시장을 바꾸는 트리거가 되었다
* 재생 에너지가 전체 전력을 초과하는 지역이 나타나면서 에너지 저장 기술이 중요해졌다
* 이 영상에서는 에너지 저장 시스템과 배터리에 대해 설명한다
* 테슬라의 일론 머스크가 이 분야에서 선도적인 역할을 하고 있다
* 친환경 발전의 안정화를 위해 배터리 장치가 필요하다는 것이 핵심 메시지이다.
* 이 영상에서는 테슬라의 역할과 전기차 산업의 발전에 대해 이야기한다
* 전기차 산업은 친환경적이고 지속 가능한 에너지를 만드는데 중요한 역할을 한다
* 테슬라는 전기차 회사로서 뿐만 아니라 스페이스와 같은 다른 비즈니스 분야에서도 활동하고 있지만, 전기차 산업의 발전을 위해 더 많은 노력이 필요하다는 생각을 갖고 있다.
* 이 영상에서는 수소 기술과 전기차의 발전에 대해 이야기한다
* 수소 연료 전지 전기차는 차 안에서 수소를 가지고 전기를 만드는 기계이다
* 수소 전기차는 앞으로 전기차의 발전 방향 중 하나일 것이다
* 이 영상에서는 수소와 관련된 기술과 산업 동향에 대해 이야기

In [17]:
# 통합 Reduce Chain
reduce_chain= (
    COMBINE_PROMPT
   | chat_llm
   | StrOutputParser() )

# 스트림 활용하기
for chunk in reduce_chain.stream(final_summary):
    print(chunk, end="", flush=True) 

이 영상에서는 전기차와 관련된 다양한 주제에 대해 다루고 있다. 최근 전 세계적으로 전기차 시장이 크게 성장하고 있으며, 현대차와 기아차가 올해의 차를 수상하며 기술력이 인정받고 있다. 테슬라는 전기차 산업에서 선도적인 역할을 하고 있으며, 수소 연료 전지 전기차도 앞으로 중요한 발전 방향 중 하나일 것이다. 전기차 생산에 사용되는 원자재 채굴과 관련된 문제와 배터리 용량과 가격 경쟁 등 다양한 이슈가 제기되고 있으며, 기업들은 이를 해결하기 위해 많은 노력을 기울이고 있다. 결론적으로, 전기차는 친환경적이며 브랜드 이미지를 바꿀 수 있는 중요한 요소이며, 전기차 시대가 더욱 가속화될 것이며, 이를 위해 기업들은 전기차 산업에 집중해야 한다는 메시지를 전한다.